# DAT158 - TMBD Box Office Prediction 

In [1]:
 from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Frame the problem and look at the big picture

I dette maskinlæringsprosjektet skal vi vi utvikle og trene en modell til å beregne inntekter til filmer. Notebooken vår er strukturert etter disse syv punktene:

1. Frame the problem and look at the big picture
2. Get the data
3. Explore the data to gain insights
4. Prepare the data to better expose the underlying data patterns to machine learning algorithms
5. Explore many different models and short-list the best ones
6. Fine-tune your models and combine them into a great solution
7. Final model and launch

## Get the data

Importerer nødvendige bibliotek

In [2]:
# Libraries

import numpy as np
import pandas as pd
pd.set_option('max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Kobler opp train og test datasettene. 

Usikker på hva det andre gjør, kopierte fra https://www.kaggle.com/artgor/eda-feature-engineering-and-model-interpretation. Måtte gjøre det for at visse ting skulle fungere.


In [3]:
train=pd.read_csv('drive/MyDrive/Skole/HVL/ML2/data/train.csv')
test=pd.read_csv('drive/MyDrive/Skole/HVL/ML2/data/test.csv')
#sampleSubmission=pd.read_csv('drive/MyDrive/Skole/HVL/ML2/data/sample_submission.csv')

test_id = test['id'].copy()

## Explore the data to gain insights

### Basic insight

Vi starter med å gjøre attributtene om fra JSON til ... 


In [4]:
dict_columns = ['belongs_to_collection', 'genres', 'production_companies',
                'production_countries', 'spoken_languages', 'Keywords', 'cast', 'crew']

def text_to_dict(df):
    for column in dict_columns:
        df[column] = df[column].apply(lambda x: {} if pd.isna(x) else ast.literal_eval(x) )
    return df
        
train = text_to_dict(train)
test = text_to_dict(test)

train.head()

NameError: ignored

Vi ønsker å få en bra oversikt over datasettene. Vi kommer til å trenge mange operasjoner for å få denne oversikten. Vi begynner beskjedent med ".info()" for å få en innsikt i hvor mange og hvilke kolonner vi har, og hvor mange non-null elementer og hvilken datatype som er i hver kolonne. 

In [ ]:
train.info()

In [ ]:
test.info()

Vi legger merke til at "test" mangler en kolonne sammenlignet med "train". Kolonnen som mangler er "revenue". Det er denne kolonnen vår maskinlæremodell skal bergene og fylle ut.

Videre ønsker vi å få en tydelig sammenliging av datasettene sine antall rader og kolonner.

In [ ]:
train.shape, test.shape

Etter vi har fått oversikt over alle kolonnene i datasettet har vi valgt å dykke dypere inn i visse attributter vi tror har større påvirkningskraft på inntektene til en film. De vi har valgt å utforske videre er belongs_to_collection, budget, genres, production_companies, production_contries og cast. 

Vi tror i tillegg at popularity, runtime og orginal_languages også vil påvirke inntekene mer enn de resterende attributtene, men vi har valgt å ikke utforske de mer. 

### Belongs to collection

Noen filmer tilhører en samling av filmer. Vi ønsker å finne ut hvor mange av filmene i datasettet som tilhører en samling og hvor mange som ikke gjør det. Denne infoen kan bli brukt senere som en **komponent/attributt/del** til å beregne inntekten til en film.

In [ ]:
train['belongs_to_collection'].apply(lambda x: len(x) if x != {} else 0).value_counts()

Det kan være vanskelig å håndtere data når de er lagret i objekter. Vi vil heller ha numeriske attributter. Vi lager to nye kolonner, collection_name og has_collection. Collection_name skal inneholde navnet til samlingen hvis filmen er en del av en samling. Has_collection skal innholde 1 eller 0, utfra om filmen er en del av en samling eller ikke. 

In [ ]:
train['collection_name'] = train['belongs_to_collection'].apply(lambda x: x[0]['name'] if x != {} else 0)
train['has_collection'] = train['belongs_to_collection'].apply(lambda x: len(x) if x != {} else 0)

test['collection_name'] = test['belongs_to_collection'].apply(lambda x: x[0]['name'] if x != {} else 0)
test['has_collection'] = test['belongs_to_collection'].apply(lambda x: len(x) if x != {} else 0)

In [ ]:
train.plot(kind="scatter", x="has_collection", y="revenue", alpha=0.4)

Vi ser at filmer som er en del av en samling tjener betydelig mer enn filmer som ikke er en del av en samling.

### Budget

Budsjettet til en film har betydning på inntekten, utfra at filmer alltid ønsker å gå i overskudd. Vi vil derfor få innsikt denne kolonnen

In [ ]:
train.budget[:10]

Som vi ser over, så har flere filmer 0 i budsjett. Vi antar at grunnen til dette er mangel på data. Vi kommer derfor til å fikse dette under punkt 4.

In [ ]:
train.plot(kind="scatter", x="budget", y="revenue", alpha=0.4)

Figuren over viser at i de fleste tilfeller er det en sammenheng mellom budsjett og inntekt.

### Genres

In [ ]:
for i, e in enumerate(train['genres'][:5]):
    print(i, e)

Filmer kan inngå i flere sjangere. Antall sjangere en film har kan ha en sammenheng i hvor suksessfull den blir. Denne informasjonen kan også bli brukt til å beregne en films inntekt. Vi skriver derfor ut hvor mange filmer som har et gitt antall sjangere.

In [ ]:
print('Number of genres in films')
train['genres'].apply(lambda x: len(x) if x != {} else 0).value_counts()

Dataen over viser at de fleste filmer har 2 eller 3 sjangere, og at det er svært få filmer som har 0, 6 eller 7 sjangere. 

Videre er vi nysgjerrige på sammenhengen mellom antall sjangere og inntekten.

In [ ]:
train['films_in_number_of_genres'] = train['genres'].apply(lambda x: len(x) if x != {} else 0).value_counts()

test['films_in_number_of_genres'] = test['genres'].apply(lambda x: len(x) if x != {} else 0).value_counts()

In [ ]:
train.plot(kind="scatter", x="films_in_number_of_genres", y="revenue", alpha=0.4)

De to prikkene lengst til høyre representerer 2 og 3 sjangere pr film, mens prikken på 600 på x-asken representerer 1 sjanger pr film. Utfra figueren ser vi at 2 og 3 sjangere pr film har en høyere inntekt enn de fleste andre, men 1 sjanger pr film er overlegen mtp inntekt.

Det vil være nyttig å vite hvor mange og hvilke sjangere vi har å jobbe med, og hvor mange filmer som inngår i hver av sjangerene. 

Vi begynner med å lagre sjangerne til hver film, for deretter å telle hvor mange filmer som inngår i hver sjanger. Til slutt visualiserer vi dataen slik at vi ser alle sjangerne og hvor populær hver sjanger er.

In [ ]:
genres_per = train['genres'].apply(lambda x: [i['name'] for i in x] if x != {} else [])
genres_per

In [ ]:
genres_count = Counter([i for j in genres_per for i in j]).most_common()
fig = plt.figure(figsize=(8, 5))
sns.barplot([val[1] for val in genres_count], [val[0] for val in genres_count])
plt.xlabel('Count')
plt.title('Top 20 genres count')
plt.show()

### Popularity

Vi vil sjekke hvordan popularity påvirker inntekten.

In [ ]:
train.plot(kind="scatter", x="popularity", y="revenue", alpha=0.4)

Det ser ikke ut som popularity har stor innvirkning på inntekten til en film.

### Production companies 

Vi skal nå se på produksjonsselskaper. Vi begynner med å få en oversikt over antall produksjonsselskaper hver film har.

In [ ]:
print('Number of production companies in films')
train['production_companies'].apply(lambda x: len(x) if x != {} else 0).value_counts()

Dataen viser at de fleste filmer har 1-3 produksjonsselskaper, og at noen filmer har 0 eller 4-6 produksjonsselskaper.

Det kan være nyttig å vite hvilke produksjonsselskaper som ofte går igjen og hvor mange filmer de har i datasettet. Vi er usikre på hvordan vi skal bruke denne dataen videre, men vi tenker at det uansett er kjekt å bli kjent med dataen

In [ ]:
list_of_companies = list(train['production_companies'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)

Counter([i for j in list_of_companies for i in j]).most_common(30)

In [ ]:
train['films_in_number_of_companies'] = train['production_companies'].apply(lambda x: len(x) if x != {} else 0).value_counts()

In [ ]:
train.plot(kind="scatter", x="films_in_number_of_companies", y="revenue", alpha=0.4)

De fire første prikkene fra høyre viser inntekten til filmer med ett, to, tre og fire produksjonsselskap i samme film. Filmene med ett produksjonsselskap ser ut til å ha høyere inntekter enn de med flere. Det ser ikke ut til å være en sammenheng mellom om en film har to eller flere produksjonsselskaper og inntekten.

### Production countries

En annen **komponent** som kan indirekte påvirke inntekten til en film er i hvilke(t) land filmen er produsert i. 

Igjen begynner vi med å få en oversikt over hvor mange land filmen er produsert i.

In [ ]:
print('Number of production countries in films')
train['production_countries'].apply(lambda x: len(x) if x != {} else 0).value_counts()

Som vi ser så blir de fleste filmene produsert i ett land, men det er god del som også blir produsert i to land.

Vi ønsker også å få en oversikt over hvilke land som har vært med å produsere filmene, samt hvor mange filmer som har blitt produsert i hvert land. Vi gjør det samme som vi gjorde med sjangere for å visualisere dataen

In [ ]:
list_of_countries = list(train['production_countries'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)

In [ ]:
production_countries_per = train['production_countries'].apply(lambda x: [i['name'] for i in x] if x != {} else [])
production_countries_per

In [ ]:
production_countries_count = Counter([i for j in production_countries_per for i in j]).most_common(20)
fig = plt.figure(figsize=(8, 5))
sns.barplot([val[1] for val in production_countries_count], [val[0] for val in production_countries_count])
plt.xlabel('Count')
plt.title('Top 20 production countries')
plt.show()

In [ ]:
train['number_of_films_in_countries'] = train['production_countries'].apply(lambda x: len(x) if x != {} else 0).value_counts()

In [ ]:
train.plot(kind="scatter", x="number_of_films_in_countries", y="revenue", alpha=0.4)

Figuren over viser at filmer som er laget i USA generelt tjener mye mer enn resterende. Vi ser også at det er et annet land som også tjener fra på sine filmer, men det landet produserer veldig få filmer, så vi går ikke mer i dybden på det.

### Runtime

In [ ]:
train.plot(kind="scatter", x="runtime", y="revenue", alpha=0.4)

Over sammenligner vi runtime oppmot revenue, og ser at filmene som tjener mest varer som regel mellom 90-150 minutter

### Cast

In [ ]:
print('Amount of casted persons in films')
train['cast'].apply(lambda x: len(x) if x != {} else 0).value_counts().head(10)

Dataen over viser at de fleste filmene har mellom 9-18 personer i casten. Antall personer i casten kan ha noe å si på kvaliteten til filmen, som igjen kan påvirke inntektene. 

## Prepare the data to better expose the underlying data patterns to machine learning algorithms

Vi ønsker å gjøre endringer på noen deler av dataen fra datasettet. Det kan være vanskelig å jobbe med objekter, derfor skal vi gjøre om flere av kolonnene til å inneholde nummeriske attributter istedenfor objekter.

I tillegg er det mange kolonner som har liten innvirkning på inntekten. Vi tenker å fjerne disse kolonnene.

### Fjerne unødvendige kolonner

In [ ]:
train = train.drop(['id', 'homepage', 'imdb_id', 'original_title', 'overview', 'poster_path', 'production_companies', 'production_countries',
                    'release_date', 'spoken_languages', 'status', 'tagline', 'title', 'Keywords', 'films_in_number_of_genres',
                    'films_in_number_of_companies', 'number_of_films_in_countries'], axis=1)

test = test.drop(['id', 'homepage', 'imdb_id', 'original_title', 'overview', 'poster_path', 'production_companies', 'production_countries',
                  'release_date', 'spoken_languages', 'status', 'tagline', 'title', 'Keywords'], axis=1)

In [ ]:
train.info()

### Belongs to collection

Siden vi la til to nye kolonner tidligere i prosjektet som skulle erstatte belongs_to_collection, så sletter vi nå belongs_to_collection. 

In [ ]:
train = train.drop(['belongs_to_collection'], axis=1)
train = train.drop(['collection_name'], axis=1)

test = test.drop(['belongs_to_collection'], axis=1)
test = test.drop(['collection_name'], axis=1)

### Genres

Vi ønsker å gjøre genres om til number_of_genres for å lagre antall sjangere hver film har.

In [ ]:
train['number_of_genres'] = train['genres'].apply(lambda x: len(x) if x != {} else 0)
test['number_of_genres'] = test['genres'].apply(lambda x: len(x) if x != {} else 0)

train = train.drop(['genres'], axis=1)
test = test.drop(['genres'], axis=1)

### Original language

Vi tror at orginalspråket til en film kan ha mye å si for inntekten. Vi tror at filmer som snakker engelsk generelt har en høyere inntekt enn filmer som ikke snakker engelsk, siden engelsk talende filmer når ut til flere. Vi skal derfor endre original_language til speaks_english. Speaks_english skal inneholde 0 eller 1 utfra om orginalspråket er engelsk eller ikke.

In [ ]:
train['speaks_english'] = train['original_language'].apply(lambda x: 1 if x == "en" else 0)
test['speaks_english'] = test['original_language'].apply(lambda x: 1 if x == "en" else 0)

train = train.drop(['original_language'], axis=1)
test = test.drop(['original_language'], axis=1)

In [ ]:
train.plot(kind="scatter", x="speaks_english", y="revenue", alpha=0.4)

Som vi ser så har filmer med orginalspråk engelsk en betydelig høyere inntekt enn de som ikke snakker engelsk. 

### Production countries

Etter at vi så på dataen til production countries 

In [ ]:
#train['made_in_us'] = train['production_countries'].apply(lambda x: 1 if "US" in next(iter('iso_3166_1')) else 0)
#train['made_in_us'] = train['production_countries'].apply(lambda x: 1 if "US" in 'iso_3166_1' else 0)
#train['made_in_us'] = train['production_countries'].apply(lambda x: 1 if x['name'] == "United States of America" else 0)

#genres_per = train['genres'].apply(lambda x: [i['name'] for i in x] if x != {} else [])

#def is_US(a):
#    if "US" in a:
#        return 1
#    else: return 0

#train["made_in_us"] = train["production_countries"].apply(lambda x: 0 if x is np.nan else is_US(x))

#train['made_in_us']
#train['production_countries']

### Runtime

Det mangler to non-null verdier i runtime. Det skal vi ordne opp i nå

In [ ]:
medianTrain = train["runtime"].median()
train["runtime"].fillna(medianTrain, inplace=True)

medianTest = test["runtime"].median()
test["runtime"].fillna(medianTest, inplace=True)

### Cast 

Istedenfor å ha lagret hele casten ønsker vi å lagre antallet som er med i casten. Det vil bli mye lettere å håndtere et spesifikt antall enn et objekt.

In [ ]:
train['cast_count'] = train['cast'].apply(lambda x: 0 if x is np.nan else len(x))
test['cast_count'] = test['cast'].apply(lambda x: 0 if x is np.nan else len(x))

train = train.drop(['cast'], axis=1)
test = test.drop(['cast'], axis=1)

In [ ]:
train.plot(kind="scatter", x="cast_count", y="revenue", alpha=0.4)

### Crew

Istedenfor å ha lagret hele crewet \ ønsker vi å lagre antallet som er med i crewet. Det vil bli mye lettere å håndtere et spesifikt antall enn et objekt.

In [ ]:
train['crew_count'] = train['crew'].apply(lambda x: 0 if x is np.nan else len(x))
test['crew_count'] = test['crew'].apply(lambda x: 0 if x is np.nan else len(x))

train = train.drop(['crew'], axis=1)
test = test.drop(['crew'], axis=1)

In [ ]:
train.plot(kind="scatter", x="crew_count", y="revenue", alpha=0.4)

In [ ]:
train.info()

### Korrelasjon

In [ ]:
corr_matrix = train.corr()
corr_matrix["revenue"].sort_values(ascending=False)

In [ ]:
revenue = train['revenue'].copy()
train = train.drop(['revenue'], axis=1)

### Transformation pipelines

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

In [ ]:
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import OneHotEncoder
num_attribs = list(train)
full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
    ])
train_tr = full_pipeline.fit_transform(train)
test_tr = full_pipeline.transform(test)

In [ ]:
train_tr.shape, test_tr.shape

## Explore many different models and short-list the best ones

### Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(train_tr, revenue)

In [ ]:
from sklearn.metrics import mean_squared_error
train_predictions = lin_reg.predict(train_tr)
lin_mse = mean_squared_error(revenue, train_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

82.965.955 er en alt for dårlig score. Vi prøver neste modell!

### Decision Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(train_tr, revenue)

In [ ]:
train_predictions = tree_reg.predict(train_tr)
tree_mse = mean_squared_error(revenue, train_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

Her ser vi at modellen har blit overfitted.

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, train_tr, revenue, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores)

13.667.788 er en mye bedre score, men vi prøver en til modell! 

### Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(train_tr, revenue)

forest_mse= mean_squared_error(revenue, train_predictions)
forest_rmse = np.sqrt(forest_mse)

scores = cross_val_score(forest_reg, train_tr, revenue, scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-scores)

display_scores(forest_rmse_scores)

8.912.464 er er akseptabel score. Vi er fornøyde med modellen! 

## Fine-tune your models and combine them into a great solution

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=8),
        'n_jobs': [-1]
    }

forest_reg = RandomForestRegressor(random_state=69)

rnd_search = RandomizedSearchCV(
                forest_reg, 
                param_distributions=param_distribs,
                n_iter=10, 
                cv=5, 
                scoring='neg_mean_squared_error', 
                random_state=69
            )

rnd_search.fit(train_tr, revenue)

In [ ]:
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

Vi ser at den beste kombinasjonen er max_feature = 3 og n_estimators = 184

In [ ]:
feature_importances = rnd_search.best_estimator_.feature_importances_
feature_importances

In [ ]:
sorted(zip(feature_importances, num_attribs), reverse=True)


## Final model and launch

In [ ]:
final_model = rnd_search.best_estimator_
final_predictions =  final_model.predict(test_tr)

In [ ]:
submission = pd.DataFrame({
    'id': test_id,
    'revenue': final_predictions
})

In [ ]:
submission.head()

In [ ]:
submission.to_csv('drive/MyDrive/Skole/HVL/ML2/data/submission.csv', index=False)

In [ ]:
from joblib import dump
dump(final_model, 'drive/MyDrive/Skole/HVL/ML2/TMDB_BO.joblib', compress=6)